# **Project - Airline AI Assistant**

## **Important Features of Airline AI Assistant**

### ✈️ **Flight Availability**
- Check available flights to a destination with:
  - Airline name, departure time, price, and duration.
- Alerts user if no flights are found.

### 🛫 **Step-by-step Flight Booking**
- Guides users through:
  1. Selecting source and destination cities.
  2. Choosing a flight option.
  3. Providing passenger details (name, age).
- Ensures source and destination are not the same.

### 🌛 **Ticket Generation**
- Creates a unique ticket file: `firstName_lastName_bookingNumber.txt`.
- Ticket includes:
  - Passenger details
  - Flight details (airline, time, price, seat number)

### 📊 **Generate Summary Report**
- Summarizes all bookings into a single file: `summary_report.txt`.
- Includes all flight and passenger details for review or administration.

### 🪑 **Automated Seat Assignment**
- Assigns a random but consistent seat number for each booking.
- Ensures unique seats for each flight.

### 💬 **Interactive Chat Interface**
- Real-time conversation via Gradio.
- Provides clear, polite responses based on user input.

### 🛠️ **Modular Tool Support**
- Integrated tools for:
  - Checking flight availability
  - Booking flights
  - Generating reports
- Easily extensible for future features.

### 🖼️ **Destination Image Generation**
- Generates a pop-art-style image of the user's destination city using AI.
- Images are displayed in real-time for enhanced user experience.
- Triggered when booking a flight or requesting a destination-specific image.

### 🔊 **Text-to-Speech (TTS) Support**
- Converts the assistant's responses into speech for accessibility.
- Offers an optional TTS feature for audio playback of replies.

### 🛡️ **Error Handling**
- Validates user inputs and prevents invalid bookings.
- Graceful error messages for smooth user experience.

---

These enhanced features ensure a seamless, user-friendly experience while booking flights, generating destination images, and interacting with FlightAI Assistant!



# Let's go multi-modal!!


In [ ]:
# !ffmpeg -version
# !ffprobe -version
# !ffplay -version

In [2]:
import os
import json
import random
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

import base64
from io import BytesIO
from PIL import Image
from pydub import AudioSegment
import subprocess
import tempfile
import time

########################################
# 1) Environment & Model Setup
########################################
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

########################################
# 2) System Prompt
########################################
system_message = (
    "You are a helpful assistant for an Airline called FlightAI.\n\n"
    "When the user wants to book a flight, follow these steps:\n"
    "1. Ask for the source city.\n"
    "2. Ask for the destination city (must be different from source).\n"
    "3. Call the function 'check_flight_availability' with the user's destination.\n"
    "   - If it returns an empty list, say: 'No flights to that city'.\n"
    "   - If it returns flights, list them EXACTLY, in a numbered list, showing airline, time, price, and duration.\n"
    "4. Wait for the user to pick one flight option by number.\n"
    "5. Then ask for passenger first name, last name, and age.\n"
    "6. Finally call 'book_flight' to confirm and show the user the real seat number and booking details.\n\n"
    "You also have a tool 'generate_report' which summarizes ALL booked tickets in a single file.\n\n"
    "IMPORTANT:\n"
    "- Always call 'check_flight_availability' if user mentions a new destination.\n"
    "- Do not invent flights or seat numbers. Use what the function calls return.\n"
    "- Source and destination cannot be the same.\n"
    "- Every time a flight is booked, produce a new ticket file named firstName_lastName_bookingNumber.txt.\n"
    "- If a city is not in flight_availability, say 'No flights found for that city'.\n"
    "If the user wants all tickets summarized, call 'generate_report' with no arguments (the function has none).\n"
    "If you don't know something, say so.\n"
    "Keep answers short and courteous.\n"
)

########################################
# 3) Flight Data & Bookings
########################################
flight_availability = {
    "london": [
        {"airline": "AirlinesAI", "time": "10:00 AM", "price": "$799",  "duration": "8 hours"},
        {"airline": "IndianAirlinesAI", "time": "3:00 PM", "price": "$899",  "duration": "8 hours"},
        {"airline": "AmericanAirlinesAI","time": "8:00 PM", "price": "$999",  "duration": "8 hours"},
    ],
    "paris": [
        {"airline": "EuropeanAirlinesAI","time": "11:00 AM","price": "$399",  "duration": "7 hours"},
        {"airline": "BudgetAirlines",    "time": "6:00 PM", "price": "$2399", "duration": "7 hours"},
    ],
    "tokyo": [
        {"airline": "TokyoAirlinesAI",   "time": "12:00 PM","price": "$4000", "duration": "5 hours"},
        {"airline": "FastFly",           "time": "7:00 PM", "price": "$1400", "duration": "5 hours"},
    ],
    "berlin": [
        {"airline": "BerlinAirlinesAI",  "time": "9:00 AM", "price": "$499",  "duration": "6 hours"},
        {"airline": "AmericanAirlinesAI","time": "4:00 PM", "price": "$899",  "duration": "6 hours"},
    ],
    "nagpur": [
        {"airline": "IndianAirlinesAI",  "time": "8:00 AM", "price": "$1000", "duration": "10 hours"},
        {"airline": "JetAirlines",       "time": "2:00 PM", "price": "$1500", "duration": "10 hours"},
        {"airline": "AirlinesAI",        "time": "10:00 PM","price": "$800",  "duration": "10 hours"},
    ],
}
flight_bookings = []

########################################
# 4) Helper Functions
########################################
def generate_seat_numbers(seed_value):
    random.seed(seed_value)
    return [
        f"{random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}{random.randint(1, 99):02}"
        for _ in range(5)
    ]

def check_flight_availability(destination_city: str):
    print(f"[TOOL] check_flight_availability({destination_city})")
    city = destination_city.lower()
    return flight_availability.get(city, [])

def generate_ticket_file(booking_dict, booking_number):
    fname = booking_dict["first_name"].replace(" ", "_")
    lname = booking_dict["last_name"].replace(" ", "_")
    filename = f"{fname}_{lname}_{booking_number}.txt"

    content = (
        "Flight Ticket\n"
        "=============\n"
        f"Booking #   : {booking_number}\n"
        f"Passenger   : {booking_dict['first_name']} {booking_dict['last_name']}, Age {booking_dict['age']}\n"
        f"Source      : {booking_dict['source']}\n"
        f"Destination : {booking_dict['destination']}\n"
        f"Airline     : {booking_dict['airline']}\n"
        f"Departure   : {booking_dict['time']}\n"
        f"Price       : {booking_dict['price']}\n"
        f"Duration    : {booking_dict['duration']}\n"
        f"Seat Number : {booking_dict['seat']}\n"
    )
    with open(filename, "w") as f:
        f.write(content)

    print(f"[TOOL] Ticket file generated => {filename}")
    return filename

def book_flight(source, destination, option_index, first_name, last_name, age):
    print(f"[TOOL] book_flight({source=}, {destination=}, {option_index=})")

    if source.lower() == destination.lower():
        return "Error: source and destination must not be the same."

    try:
        idx = int(option_index)
    except ValueError:
        return "Error: flight option number is not a valid integer."

    flights = check_flight_availability(destination)
    if not flights:
        return f"Error: No flights found for {destination.title()}."

    pick = idx - 1
    if pick < 0 or pick >= len(flights):
        return f"Error: Invalid flight option #{idx} for {destination.title()}."

    chosen_flight = flights[pick]
    airline   = chosen_flight["airline"]
    dep_time  = chosen_flight["time"]
    price     = chosen_flight["price"]
    duration  = chosen_flight["duration"]

    seat_list = generate_seat_numbers(hash(destination + airline + str(len(flight_bookings))))
    chosen_seat = seat_list[0]

    new_booking = {
        "source":      source.title(),
        "destination": destination.title(),
        "airline":     airline,
        "time":        dep_time,
        "price":       price,
        "duration":    duration,
        "seat":        chosen_seat,
        "first_name":  first_name.title(),
        "last_name":   last_name.title(),
        "age":         age,
    }
    flight_bookings.append(new_booking)

    booking_number  = len(flight_bookings)
    ticket_filename = generate_ticket_file(new_booking, booking_number)

    confirmation = (
        f"Booking #{booking_number} confirmed for {first_name.title()} {last_name.title()}. "
        f"Flight from {source.title()} to {destination.title()} on {airline} at {dep_time}. "
        f"Ticket saved to {ticket_filename}."
    )
    print(f"[TOOL] {confirmation}")
    return confirmation

def generate_report():
    print(f"[TOOL] generate_report called.")
    report_content = "Flight Booking Summary Report\n"
    report_content += "=============================\n"

    if not flight_bookings:
        report_content += "No bookings found.\n"
    else:
        for i, booking in enumerate(flight_bookings, start=1):
            report_content += (
                f"Booking #   : {i}\n"
                f"Passenger   : {booking['first_name']} {booking['last_name']}, Age {booking['age']}\n"
                f"Source      : {booking['source']}\n"
                f"Destination : {booking['destination']}\n"
                f"Airline     : {booking['airline']}\n"
                f"Departure   : {booking['time']}\n"
                f"Price       : {booking['price']}\n"
                f"Duration    : {booking['duration']}\n"
                f"Seat Number : {booking['seat']}\n"
                "-------------------------\n"
            )

    filename = "summary_report.txt"
    with open(filename, "w") as f:
        f.write(report_content)

    msg = f"Summary report generated => {filename}"
    print(f"[TOOL] {msg}")
    return msg

########################################
# Image & Audio
########################################
def artist(city):
    print(f"[artist] Generating an image for {city}")
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"An image representing a vacation in {city}, showing tourist spots in a vibrant pop-art style",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(1)
        subprocess.call(
            ["ffplay", "-nodisp", "-autoexit", "-hide_banner", temp_path],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
    finally:
        try:
            os.remove(temp_path)
        except:
            pass

def talker(message):
    print("[talker] Generating TTS for final reply...")
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # or "alloy"
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

########################################
# 5) Tools JSON Schemas
########################################
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket for the city from flight list data.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "City name.",
            },
        },
        "required": ["destination_city"],
    },
}

availability_function = {
    "name": "check_flight_availability",
    "description": (
        "Check flight availability for the specified city. "
        "Returns a list of {airline, time, price, duration}."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "City name to check in flight_availability dict.",
            },
        },
        "required": ["destination_city"],
    },
}

book_function = {
    "name": "book_flight",
    "description": (
        "Book a flight using an option index for the chosen city. "
        "Generates a unique ticket file firstName_lastName_{bookingNumber}.txt each time."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "source": {
                "type": "string",
                "description": "User's source city (must differ from destination).",
            },
            "destination": {
                "type": "string",
                "description": "User's destination city.",
            },
            "option_index": {
                "type": "string",
                "description": "1-based flight option number the user selected from check_flight_availability.",
            },
            "first_name": {
                "type": "string",
                "description": "Passenger's first name.",
            },
            "last_name": {
                "type": "string",
                "description": "Passenger's last name.",
            },
            "age": {
                "type": "string",
                "description": "Passenger's age.",
            },
        },
        "required": ["source", "destination", "option_index", "first_name", "last_name", "age"],
    },
}

report_function = {
    "name": "generate_report",
    "description": (
        "Generates a summary report of ALL tickets in summary_report.txt."
    ),
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
    },
}

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": availability_function},
    {"type": "function", "function": book_function},
    {"type": "function", "function": report_function},
]

########################################
# 6) Handling the Tool Calls
########################################
def handle_tool_call(message):
    """
    Called if LLM tries to call one of the 4 tools.
    If the tool is 'book_flight', we also capture the 'destination' so we can
    possibly generate an image for it if the user wants that.
    """
    tool_call = message.tool_calls[0]
    fn_name   = tool_call.function.name
    args      = json.loads(tool_call.function.arguments)

    # We'll keep track of the 'dest_city' if the function is 'book_flight'
    # so we can generate an image for that city if the user wants to.
    dest_city = None

    if fn_name == "get_ticket_price":
        city = args.get("destination_city")
        flights = check_flight_availability(city)
        if not flights:
            response_content = {"destination_city": city, "price": "No flights found."}
        else:
            response_content = {
                "destination_city": city,
                "price": flights[0]["price"]
            }

    elif fn_name == "check_flight_availability":
        city = args.get("destination_city")
        flights = check_flight_availability(city)
        response_content = {"destination_city": city, "availability": flights}

    elif fn_name == "book_flight":
        src  = args.get("source")
        dest = args.get("destination")
        idx  = args.get("option_index")
        fnam = args.get("first_name")
        lnam = args.get("last_name")
        age  = args.get("age")

        confirmation = book_flight(src, dest, idx, fnam, lnam, age)
        response_content = {
            "source": src,
            "destination": dest,
            "option_index": idx,
            "first_name": fnam,
            "last_name":  lnam,
            "age":        age,
            "confirmation": confirmation
        }
        dest_city = dest  # track for potential image generation

    elif fn_name == "generate_report":
        msg = generate_report()
        response_content = {"report": msg}

    else:
        response_content = {"error": f"Unknown tool: {fn_name}"}

    return {
        "role": "tool",
        "content": json.dumps(response_content),
        "tool_call_id": tool_call.id,
    }, dest_city



OpenAI API Key exists and begins sk-proj-


In [4]:
########################################
# 7) The Chat Function
########################################
def chat(history, do_tts=True, do_image=True):
    """
    Takes conversation history, returns (new_history, optional_image).
    do_tts: If True, we do talker() for the final text.
    do_image: If True, we generate an image for the destination city if book_flight was called.

    1) We'll parse the entire conversation so far.
    2) We'll see if LLM calls any tool. If tool == 'book_flight', we capture 'destination' for an image.
    3) We'll produce final text. If do_tts=True, we call talker(final_text).
    4) If do_image=True and we have a 'destination' from book_flight, we call artist(destination).
    """
    messages = [{"role": "system", "content": system_message}] + history
    image_to_return = None
    booked_destination = None

    try:
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

        # If the LLM calls a tool:
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            print(f"[INFO] Tool call requested: {msg.tool_calls[0]}")
            
            tool_response, dest_city = handle_tool_call(msg)
            print(f"[INFO] Tool response: {tool_response}")

            # If it's 'book_flight' or we got a 'dest_city', store it 
            if dest_city:
                booked_destination = dest_city

            messages.append(msg)
            messages.append(tool_response)

            # Re-send updated conversation
            response = openai.chat.completions.create(model=MODEL, messages=messages)

        final_text = response.choices[0].message.content

        # # TTS
        # if do_tts:
        #     talker(final_text)

        # If user wants images and we have a 'book_flight' city:
        if do_image and booked_destination:
            image_to_return = artist(booked_destination)

        new_history = history + [{"role": "assistant", "content": final_text}]

                # TTS
        if do_tts:
            talker(final_text)

        return new_history, image_to_return

    except Exception as e:
        print(f"[ERROR] {e}")
        error_msg = "I'm sorry, something went wrong while processing your request."
        new_history = history + [{"role":"assistant","content":error_msg}]
        return new_history, None

########################################
# 8) Enhanced Gradio UI (Side-by-Side)
########################################
with gr.Blocks() as ui:
    gr.Markdown("## FlightAI Assistant (Flexible Images + TTS)")

    with gr.Row():
        with gr.Column(scale=1):
            chatbot = gr.Chatbot(
                label="FlightAI Chat",
                height=500,
                type="messages",
            )
        with gr.Column(scale=1):
            image_output = gr.Image(
                label="Generated Image (if any)",
                height=500
            )
    
    # Next row: user input + checkboxes for TTS + images
    user_input = gr.Textbox(
        label="Type your message here...",
        placeholder="Ask about flights, destinations, etc.",
    )

    with gr.Row():
        enable_tts_checkbox = gr.Checkbox(label="Enable TTS (Audio)?", value=False)
        enable_image_checkbox = gr.Checkbox(label="Generate Destination Image? ($0.05 each)", value=False)
        clear_btn = gr.Button("Clear Conversation")

    ############################################################################
    # Logic for handling user messages
    ############################################################################
    def user_message(message, chat_history):
        chat_history += [{"role": "user", "content": message}]
        return "", chat_history

    def process_chat(chat_history, do_tts, do_image):
        new_history, maybe_image = chat(chat_history, do_tts=do_tts, do_image=do_image)
        return new_history, maybe_image

    # 1) Submitting the user input
    user_input.submit(
        fn=user_message,
        inputs=[user_input, chatbot],
        outputs=[user_input, chatbot]
    ).then(
        fn=process_chat,
        inputs=[chatbot, enable_tts_checkbox, enable_image_checkbox],
        outputs=[chatbot, image_output]
    )

    # 2) Clear conversation
    def clear_conv():
        return [], None

    clear_btn.click(
        fn=clear_conv,
        outputs=[chatbot, image_output],
        queue=False
    )

ui.launch()


* Running on local URL:  http://127.0.0.1:7916

To create a public link, set `share=True` in `launch()`.


[talker] Generating TTS for final reply...
[talker] Generating TTS for final reply...
[INFO] Tool call requested: ChatCompletionMessageToolCall(id='call_vKggZD9JxH5QnfsXfNehdhIn', function=Function(arguments='{"destination_city":"London"}', name='check_flight_availability'), type='function')
[TOOL] check_flight_availability(London)
[INFO] Tool response: {'role': 'tool', 'content': '{"destination_city": "London", "availability": [{"airline": "AirlinesAI", "time": "10:00 AM", "price": "$799", "duration": "8 hours"}, {"airline": "IndianAirlinesAI", "time": "3:00 PM", "price": "$899", "duration": "8 hours"}, {"airline": "AmericanAirlinesAI", "time": "8:00 PM", "price": "$999", "duration": "8 hours"}]}', 'tool_call_id': 'call_vKggZD9JxH5QnfsXfNehdhIn'}
[talker] Generating TTS for final reply...
[INFO] Tool call requested: ChatCompletionMessageToolCall(id='call_hZZr3hJWUvU6NzrdCXdFMNxL', function=Function(arguments='{"source":"Melbourne","destination":"London","option_index":"3","first_name"